In [30]:
#Mocap Arena Camera Calibrate Program
#Mateus Pincho - calibrate intrisics using OpenCV
from glob import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [31]:
#Reconstrução do tabuleiro
def construct3DPoints(patternSize,squareSize):
    X = np.zeros((patternSize[0]*patternSize[1],3), np.float32)
    X[:,:2] = np.mgrid[0:patternSize[0],0:patternSize[1]].T.reshape(-1,2)
    X = X * squareSize # Square size não interfere na calibração -> somente um fator de escala
                       # o square size não é utilizado na calibração dos cara de stanford
    return X

#Detectando os corners
def detectCorners(images, boardPoints, patternSize):
    worldPoints = []
    imagePoints = [] 

    img_size = 0
    counter = 0
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_size = gray.shape[::-1]
        ret, corners = cv2.findChessboardCornersSB(gray, patternSize, None)
        if ret == True:
            print("Corners found in image " + str(fname)) #- see if corners are found 
            #corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
            worldPoints.append(boardPoints)
            imagePoints.append(corners)
            counter+=1

    print("Corners found in " + str(counter) + " images")
    #print(img_size)
    return worldPoints, imagePoints, img_size

#Descobrindo os intrisicos, extrinsecos e o erro de reprojeção 
def calibrate(images, useFisheye, patternSize, squareSize):

    boardPoints = construct3DPoints(patternSize, squareSize)

    worldPoints, imagePoints, imgSize = detectCorners(images, boardPoints, patternSize)

    if useFisheye:
        flagsCalib = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_FIX_SKEW+cv2.fisheye.CALIB_CHECK_COND
        calibrateCriteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER,30, 1e-12)

        ret, cameraMatrix, k, R, t = cv2.fisheye.calibrate(np.expand_dims(np.asarray(worldPoints), -2), imagePoints, imgSize, None, None,
                                                                    flags=flagsCalib,criteria=calibrateCriteria)
    else:
        flagsCalib = cv2.CALIB_RATIONAL_MODEL

        ret, cameraMatrix, k, rvecs, tvecs, stdIntrinc, stdExtrinsic, perViewError = cv2.calibrateCameraExtended(worldPoints, imagePoints, imgSize, None, None,
                                                                flags=flagsCalib)

    print("RMS re-projection error:", ret)
    print("The median re-projection error", np.median(perViewError))
    print("Camera Matrix:\n", cameraMatrix)
    print("Distortion Parameters:\n", k)

    return cameraMatrix, k, rvecs, tvecs, stdIntrinc, stdExtrinsic, perViewError

In [32]:
dataset    =  ['../../images/virtual/set_5/*.jpg',  # dataset[0]
              '../../images/virtual/set_10/*.jpg',  # dataset[1]
              '../../images/virtual/set_15/*.jpg',  # dataset[2]
              '../../images/virtual/set_20/*.jpg',  # dataset[3]
              '../../images/virtual/set_25/*.jpg']  # dataset[4]

In [33]:
# escolha o conjunto de imagens
image_set = glob(dataset[1])

# defina os parâmetros da calibração
pattern_size = (7,7)
square_size = 30
useFisheye = False

In [34]:
camera_matrix, distortion, rotation, translation, _, _, _ = calibrate(image_set, useFisheye, pattern_size, square_size)

Corners found in image ../../images/virtual/set_10/image10.jpg
Corners found in image ../../images/virtual/set_10/image4.jpg
Corners found in image ../../images/virtual/set_10/image8.jpg


Corners found in image ../../images/virtual/set_10/image1.jpg
Corners found in image ../../images/virtual/set_10/image9.jpg
Corners found in image ../../images/virtual/set_10/image6.jpg
Corners found in image ../../images/virtual/set_10/image7.jpg
Corners found in image ../../images/virtual/set_10/image2.jpg
Corners found in image ../../images/virtual/set_10/image3.jpg
Corners found in image ../../images/virtual/set_10/image5.jpg
Corners found in 10 images
RMS re-projection error: 0.26582710684439875
The median re-projection error 0.2427068972102226
Camera Matrix:
 [[415.49498276   0.         239.65111064]
 [  0.         415.46007245 238.55385495]
 [  0.           0.           1.        ]]
Distortion Parameters:
 [[-1.40439541e+01 -2.18639606e+01 -8.47167306e-04 -4.93616468e-04
   7.92350158e+02 -1.40656846e+01 -2.14351927e+01  7.90053531e+02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [35]:
# Mean Error

rvecs = []
tvecs = []
errors = []

board_points = construct3DPoints(pattern_size, square_size)

for image in image_set:
    img = cv2.imread(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCornersSB(gray, pattern_size, None)
    _, rvec, tvec = cv2.solvePnP(board_points, corners, camera_matrix, 0)
    rvecs.append(rvec)
    tvecs.append(tvecs)

    image_points, _ = cv2.projectPoints(board_points, rvec, tvec, camera_matrix, 0)

    error = cv2.norm(corners, image_points, normType= cv2.NORM_L2) / len(image_points)
    errors.append(error)

    print(f"Mean Error of Image {image}:", error, "px")

print("Median error: ", np.median(errors))

Mean Error of Image ../../images/virtual/set_10/image10.jpg: 0.03469918832396297 px
Mean Error of Image ../../images/virtual/set_10/image4.jpg: 0.032896305007500085 px
Mean Error of Image ../../images/virtual/set_10/image8.jpg: 0.03200373419060967 px
Mean Error of Image ../../images/virtual/set_10/image1.jpg: 0.03843584868272413 px
Mean Error of Image ../../images/virtual/set_10/image9.jpg: 0.05405853188913834 px
Mean Error of Image ../../images/virtual/set_10/image6.jpg: 0.03224344376830129 px
Mean Error of Image ../../images/virtual/set_10/image7.jpg: 0.05214121084900289 px
Mean Error of Image ../../images/virtual/set_10/image2.jpg: 0.030879152946113524 px
Mean Error of Image ../../images/virtual/set_10/image3.jpg: 0.03718199001073157 px
Mean Error of Image ../../images/virtual/set_10/image5.jpg: 0.02948553551028449 px
Median error:  0.03379774666573153


In [36]:
# erro RMS
from math import sqrt
rvecs = []
tvecs = []
errors = []

board_points = construct3DPoints(pattern_size, square_size)

for image in image_set:
    img = cv2.imread(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCornersSB(gray, pattern_size, None)
    _, rvec, tvec = cv2.solvePnP(board_points, corners, camera_matrix, 0)
    rvecs.append(rvec)
    tvecs.append(tvecs)

    image_points, _ = cv2.projectPoints(board_points, rvec, tvec, camera_matrix, 0)

    error = cv2.norm(corners, image_points, normType= cv2.NORM_L2) / sqrt(len(image_points))
    errors.append(error)

    print(f"RMS Error of Image {image}:", error, "px")

print("Median RMS error: ", np.median(errors))

RMS Error of Image ../../images/virtual/set_10/image10.jpg: 0.2428943182677408 px
RMS Error of Image ../../images/virtual/set_10/image4.jpg: 0.23027413505250058 px
RMS Error of Image ../../images/virtual/set_10/image8.jpg: 0.2240261393342677 px
RMS Error of Image ../../images/virtual/set_10/image1.jpg: 0.2690509407790689 px
RMS Error of Image ../../images/virtual/set_10/image9.jpg: 0.3784097232239684 px
RMS Error of Image ../../images/virtual/set_10/image6.jpg: 0.22570410637810898 px
RMS Error of Image ../../images/virtual/set_10/image7.jpg: 0.36498847594302025 px
RMS Error of Image ../../images/virtual/set_10/image2.jpg: 0.21615407062279468 px
RMS Error of Image ../../images/virtual/set_10/image3.jpg: 0.260273930075121 px
RMS Error of Image ../../images/virtual/set_10/image5.jpg: 0.20639874857199142 px
Median RMS error:  0.2365842266601207
